In [ ]:
import requests
import time
import csv
import matplotlib
import matplotlib.pyplot as plt

In [4]:
BANK_SERVICE_URL = "https://latest-960957615762.me-central1.run.app"
BALANCE_ENDPOINT = f"{BANK_SERVICE_URL}/getbalance"
LOGS_ENDPOINT = f"{BANK_SERVICE_URL}/getlogs"
NUMBER_OF_REQUESTS = 100
REQUEST_TIMEOUT = 10
DELAY_BETWEEN_REQUESTS = 0.1

In [5]:
def invoke(): # invoking the banking service 100 times to generate log data
    print(f"[1/3] Invoking {NUMBER_OF_REQUESTS} times")
    cnt = 0
    
    for i in range(NUMBER_OF_REQUESTS):
        try:
            response = requests.get(BALANCE_ENDPOINT, timeout=REQUEST_TIMEOUT)
            print(f"Request № {i+1} out of {NUMBER_OF_REQUESTS}: Status {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Request № {i+1} out of {NUMBER_OF_REQUESTS}: ERROR: ({type(e).__name__})")
        
        cnt += 1
        time.sleep(DELAY_BETWEEN_REQUESTS)

    print(f"{cnt} Requests are DONE!")

In [6]:
def fetch_logs():
    print("[2/3] Fetching logs...")
    try:
        resp = requests.get(LOGS_ENDPOINT, timeout=10)
        resp.raise_for_status()
        return resp.text
    except Exception as e:
        print(f"Failed to fetch logs: {e}")
        return ""

In [ ]:
def parse_logs(log_text):
    """Parse logs to count requests and errors."""
    lines = [line.strip() for line in log_text.strip().split('\n') if line.strip()]
    
    total_requests = sum(1 for line in lines if "Processing GET request" in line)
    errors_500 = sum(1 for line in lines if "ERROR Internal Server Error [500]" in line)
    errors_db = sum(1 for line in lines if "ERROR Database Connection Pool Empty" in line)
    warnings = sum(1 for line in lines if "WARNING Server Latency is" in line)
    
    # Warnings are NOT failures — ignore them for reliability calculation
    failures = errors_500 + errors_db
    successes = max(0, total_requests - failures) # avoiding from negative
    
    return total_requests, errors_500, errors_db, successes, warnings